In [20]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,Dense,Embedding
import re

In [3]:
data = pd.read_csv('swiggy.csv')
data.head()

,ID,Area,City,Restaurant Price,Avg Rating,Total Rating,Food Item,Food Type,Delivery Time,Review
0,1,Suburb,Ahmedabad,600,4.2,6198,Sushi,Fast Food,30-40 min,"Good, but nothing extraordinary."
1,2,Business District,Pune,200,4.7,4865,Pepperoni Pizza,Non-Vegetarian,50-60 min,"Good, but nothing extraordinary."
2,3,Suburb,Bangalore,600,4.7,2095,Waffles,Fast Food,50-60 min,Late delivery ruined it.
3,4,Business District,Mumbai,900,4.0,6639,Sushi,Vegetarian,50-60 min,Best meal I've had in a while!
4,5,Tech Park,Mumbai,200,4.7,6926,Spring Rolls,Gluten-Free,20-30 min,Mediocre experience.


In [5]:
data.describe()

,ID,Restaurant Price,Avg Rating,Total Rating
count,8000.00000,8000.000000,8000.000000,8000.000000
mean,4000.50000,544.587500,4.129900,4979.977500
std,2309.54541,287.968871,0.645791,2877.285148
min,1.00000,100.000000,3.000000,51.000000
25%,2000.75000,300.000000,3.500000,2476.000000
50%,4000.50000,500.000000,4.200000,4989.500000
75%,6000.25000,800.000000,4.700000,7498.000000
max,8000.00000,1000.000000,5.000000,10000.000000


In [8]:
data['Review'] = data['Review'].str.lower()
data['Review'] = data['Review'].replace(r'[^a-z0-9\s]','',regex=True)

data['Sentiment'] = data['Avg Rating'].apply(lambda x:1 if x>3.5 else 0)
data = data.dropna()

In [13]:
max_features = 5000
max_length = 200
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data["Review"])
X = pad_sequences(tokenizer.texts_to_sequences(data["Review"]),maxlen = max_length)
y = data["Sentiment"].values

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [18]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=16, input_length=max_length),
    SimpleRNN(64, activation='tanh', return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [27]:
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5961 - accuracy: 0.7160 - val_loss: 0.5961 - val_accuracy: 0.7156
Epoch 2/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5962 - accuracy: 0.7160 - val_loss: 0.5961 - val_accuracy: 0.7156
Epoch 3/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5959 - accuracy: 0.7160 - val_loss: 0.5964 - val_accuracy: 0.7156
Epoch 4/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5956 - accuracy: 0.7160 - val_loss: 0.5963 - val_accuracy: 0.7156
Epoch 5/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5958 - accuracy: 0.7160 - val_loss: 0.5957 - val_accuracy: 0.7156
Epoch 6/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5966 - accuracy: 0.7160 - val_loss: 0.5963 - val_accuracy: 0.7156
Epoch 7/15
180/180 [==============================] - 3s 14ms/step - loss: 0.5960 - accuracy: 0.7160 - val_loss: 0.5961 - val_accuracy: 0.7156

In [31]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"


sample_review = "Waste Experinece"
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: Waste Experinece
1/1 [==============================] - 0s 17ms/step
Sentiment: Positive (Probability: 0.71)
